In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from pathlib import Path

import tt
import bbedit

LOG = logging.getLogger(__name__)
tt.logging_init()

In [3]:
CATEGORIES = [
    "chicken",
    "cow",
    "creeper",
    "enderman",
    "pig",
    "sheep",
    "skeleton",
    "spider",
    "zombie",
]
CATEGORIES.sort()


In [ ]:
bbedit.DEBUG.clear_output()
bbedit.DEBUG

# Mobs1

In [ ]:
mobs_path = Path.home() / "src/data/mobs1"
images_path = mobs_path / "images"
bbe1 = bbedit.BBoxEdit(mobs_path / "bboxes.json")
bbe1.display()

# Mobs2

In [ ]:
mobs_path = Path.home() / "src/data/mobs2"
images_path = mobs_path / "640"
bbe2 = bbedit.BBoxEdit(mobs_path / "corrected_bboxes.json")
bbe2.display()

# Merge

In [ ]:
import shutil
import bb

base_dir = Path.home() / "src/data"
input_dirs = [base_dir / "mobs1", base_dir / "mobs2"]
output_dir = base_dir / "mobs_merged"

BBOX_FILE = "bboxes.json"
IMAGE_DIR = "images"


def merge(input_dirs, output_dir):
    output_dir.mkdir(parents=True, exist_ok=True)
    images_subdir = output_dir / IMAGE_DIR
    images_subdir.mkdir(parents=True, exist_ok=True)
    images_rel = Path(IMAGE_DIR)

    new_dset = bb.Dataset(file_path=output_dir / BBOX_FILE)

    categories = None
    for input_dir in input_dirs:
        dset = bb.Dataset.load(input_dir / BBOX_FILE)
        if categories is None:
            categories = dset.categories
            new_dset.categories = categories
        assert categories == dset.categories, f"Categories-Mismatch {input_dir}"
        for ir in dset.iter_images():
            fpath = Path(ir.file)
            new_ir = ir.model_copy(update={"file": str(images_rel / fpath.name)}, deep=True)
            print(fpath, images_subdir)
            new_dset.images.append(new_ir)
    print(new_dset)
    new_dset.save(output_dir / BBOX_FILE)

merge(input_dirs, output_dir)

In [ ]:
from pathlib import Path
import bbedit
mobs_path = Path.home() / "src/data/mobs_merged"
images_path = mobs_path / "images"
bbe2 = bbedit.BBoxEdit(mobs_path / "bboxes.json")
bbe2.display()